<p style="font-weight:bold;"> <span style="font-size: 36px"> Scenario Import Test </span> </p>

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [0]:
DataSource.Reset(x => x.ResetType<RawVariable>().ResetType<IfrsVariable>());

In [0]:
var actualEstimateTypes = new[] {"A","AA","OA","DA"};
var argsBestEstimate = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsTestScenario = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "Test", ImportFormats.Cashflow);

# Context

In [0]:
await Import.FromFile("../Files/TransactionalData/NominalCashflows_CH_2020_12.csv").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromFile("../Files/TransactionalData/Actuals_CH_2020_12.csv").WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
var ivsBestEstimateInit = await DataSource.Query<IfrsVariable>().ToArrayAsync();

# Scenario Cash flows

In [0]:
var cashflowsScenarioTest = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,CL,C,,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,CU,CL,C,,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-3.3
DT1.1,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";

In [0]:
var cashflowsScenarioTestEqualToBestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0
DT1.1,NIC,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,CU,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5";

In [0]:
await Import.FromString(cashflowsScenarioTest).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsTestScenario);
var ivsScenarioInitCashFlows = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioInitCashFlows.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());
diffs.Where(x => actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().Be(0);
diffs.Where(x => !actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);

# Scenario Actuals

In [0]:
var actualsScenarioTest = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,-308";

In [0]:
var actualsScenarioTestEqualToBestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,Test
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,-280";

In [0]:
await Import.FromString(actualsScenarioTest).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsTestScenario);
var ivsScenarioInitActuals = DataSource.Query<IfrsVariable>().ToArray();
var diffs = ivsScenarioInitActuals.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());
diffs.Where(x => actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);
diffs.Where(x => x.EstimateType == "CU").ToArray().Length.Should().Be(0);

# Robustness: back to Best Estimate

In [0]:
await Import.FromString(actualsScenarioTestEqualToBestEstimate).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromString(cashflowsScenarioTestEqualToBestEstimate).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
var ivsScenarioReimportEqualToBestEstimate = await DataSource.Query<IfrsVariable>().ToArrayAsync(); 
ivsScenarioReimportEqualToBestEstimate.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance()).Count().Should().Be(0);

# Reimport of Best Estimate Cashflow

In [0]:
var newCashflowsBestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,210,0,0,210,0,0,210,0,0,210,0,0,0,210,0,0,210,0,0,210,0,0,210,0
DT1.1,NIC,CL,C,,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37,-37
DT1.1,CU,CL,C,,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-21,-17,-17,-17,-17,-17,-17,-17,-17,-17,-17,-3.3
DT1.1,RA,CL,C,,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42,-42";

In [0]:
await Import.FromString(newCashflowsBestEstimate).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateReimportCashFlows = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsBestEstimateReimportCashFlows.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());
var items = diffs.Where(x => actualEstimateTypes.Contains(x.EstimateType)).Select(x => x.EstimateType).Distinct().ToArray();
items.Length.Should().Be(1);
(items.First() == "DA").Should().Be(true);
diffs.Where(x => !actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsTestScenario);
var ivsScenarioReimportCashFlows = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioReimportCashFlows.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());
var items = diffs.Where(x => actualEstimateTypes.Contains(x.EstimateType)).Select(x => x.EstimateType).Distinct().ToArray();
diffs.Where(x => !actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);

In [0]:
items.Length.Should().Be(1);
(items.First() == "DA").Should().Be(true);

# Reimport of Best Estimate Actuals

In [0]:
var newActualsBestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,-2000";

In [0]:
await Import.FromString(newActualsBestEstimate).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateReimportActuals = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsBestEstimateReimportActuals.Except(ivsBestEstimateReimportCashFlows, IfrsVariableComparer.Instance());
diffs.Where(x => actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);
diffs.Where(x => x.EstimateType == "CU").ToArray().Length.Should().Be(0);

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsTestScenario);
var ivsScenarioReimportActuals = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioReimportActuals.Except(ivsBestEstimateReimportCashFlows, IfrsVariableComparer.Instance());
diffs.Where(x => actualEstimateTypes.Contains(x.EstimateType)).ToArray().Length.Should().NotBe(0);
diffs.Where(x => x.EstimateType == "CU").ToArray().Length.Should().Be(0);